In [1]:
from pymongo import MongoClient
import pprint
import pandas as pd

In [2]:
client = MongoClient('ec2-54-173-229-9.compute-1.amazonaws.com:27026')
db = client.gdelt
events = db.events

In [3]:
result = events.count()
print(result)

1463717


In [4]:
import time
t0 = time.time()
d = events.aggregate([{
   "$project": {
       "Day": "$Day",
       "Actor2CountryCode": "$Actor2CountryCode",
       "Actor1CountryCode": "$Actor1CountryCode",
       "GoldsteinScale": "$GoldsteinScale",
       "latitude" : "$Actor1Geo_Lat",
       "longitude" : "$Actor1Geo_Long",
       "typeAvis":
           {"$cond": [ { "$gte": [ "$GoldsteinScale", 0 ] }, "pos", "neg" ]}}},{
   "$match": {
       "Actor2CountryCode": "FRA",
       "Day":{"$lt": 20170522, "$gte": 20170421}}},{
   "$group":{
           "_id":{"dd":"$Day", "cc":"$Actor1CountryCode", "typeAvis":"$typeAvis"},
           "lat":{"$avg": "$latitude"},
           "long":{"$avg": "$longitude"},
           "avgGS":{"$avg": "$GoldsteinScale"}, "minGS":{"$min": "$GoldsteinScale"},
           "maxGS":{"$max": "$GoldsteinScale"}, "numGS":{"$sum":1},
           "stdDevGS": {"$stdDevPop": "$GoldsteinScale" }}}, {
   "$sort": { "Day": 1, "numGS": -1 } }])
t1 = time.time()
print("TIME :", t1-t0)

TIME : 1.6304187774658203


In [5]:
df = pd.DataFrame(list(d))
df1 = pd.DataFrame(df['_id'].tolist())
df[['Country_Code','Day','Avis']] = df1
del df['_id']
df['Date'] = pd.to_datetime(df['Day'], format='%Y%m%d')
del df['Day']
df = df.sort_values(by='Date')
df = df.dropna()

In [6]:
countries = pd.read_excel('countries.xlsx')

In [7]:
for cc,lat,long in zip(countries['Code'],countries['lat'],countries['long']):
    df.loc[df['Country_Code'] == cc,'lat'] = lat
    df.loc[df['Country_Code'] == cc,'long'] = long

In [8]:
df.to_csv('data.csv',sep=';')